In [1]:
import pandas as pd

In [2]:
def custom_compare(x, y):
    if str(x) != str(y):
        raise RuntimeError(f'Ожидаемое значение: {y}. Фактическое: {x}')

# Предобработка данных

In [39]:
def parse_user_action_solution(line):
    """
    Принимает строку в обозначенном формате, а возвращает словарь.
    Ключами в словаре являются строки 'id', 'action' и 'time',
    а значениями — соответствующие ключам значения из обрабатываемой строки.
    
    Аргументы:
        line: Строка, которую необходимо превратить в словарь.
        
    Возвращаемое значение:
        Словарь в обозначенном формате.
    """
    parsed = {}
    for l in line[1:-1].split(', '):
        name, value = l.replace("'", "").split(': ')
        if name == 'id':
            value = int(value)
        elif name == 'time':
            value = pd.to_datetime(value)
        parsed[name] = value
    return parsed

In [40]:
def parse_user_action_test():
    example_1_line = "('id': 7281910, 'action': 'click', 'time': '2023-01-01 12:23:54')"
    example_1_res = {
        'id': 7281910,
        'action': 'click',
        'time': pd.Timestamp(year=2023, month=1, day=1, hour=12, minute=23, second=54),
    }
    
    custom_compare(parse_user_action_solution(example_1_line), example_1_res)
    
    example_2_line = "('id': 1, 'action': 'search', 'time': '1997-12-31 23:23:59')"
    example_2_res = {
        'id': 1,
        'action': 'search',
        'time': pd.Timestamp(year=1997, month=12, day=31, hour=23, minute=23, second=59),
    }
    
    custom_compare(parse_user_action_solution(example_2_line), example_2_res)
    
    print('Все тесты прошли успешно!')

In [41]:
parse_user_action_test()

Все тесты прошли успешно!


# Предобработка данных в pandas

In [51]:
user_session = pd.read_csv('datasets/sirius/user_session_sample.csv')

## Задание 1

Подсчитайте, сколько пропущенных значений содержится в каждой из колонок таблицы.

In [52]:
user_session.isna().sum()

id                 0
user_id        66371
ip                 0
action_time    13922
action             0
dtype: int64

## Задание 2

Известно, что значение в колонке «action_time» должно совпадать со значением `action_time`, которое хранится внутри значений характеристики «action».

Воспользуйтесь реализованной ранее функцией `parse_user_action_solution`, чтобы восстановить на основе значений из колонки «action» отсутствующую информацию о моментах времени, когда пользователи совершали соответствующие действия.

В качестве ответа укажите число пользовательских действий, которые были совершены в августе 2023-го года, рассчитанное по таблице с восстановленными данными.

In [53]:
user_session.head(3)

,id,user_id,ip,action_time,action
0,619,NaN,244.143.138.0,2021-01-20 18:36:32,"('id': 234676, 'action': 'search', 'time': '20..."
1,919428,30083.0,157.169.233.89,2023-12-10 04:13:50,"('id': 541654, 'action': 'search', 'time': '20..."
2,630539,NaN,160.111.52.224,2023-09-26 04:09:44,"('id': 230556, 'action': 'click', 'time': '202..."


In [54]:
user_session.action_time = pd.to_datetime(user_session.action_time)
user_session.action = user_session.action.apply(parse_user_action_solution)

In [55]:
user_session.head(3)

,id,user_id,ip,action_time,action
0,619,NaN,244.143.138.0,2021-01-20 18:36:32,"{'id': 234676, 'action': 'search', 'time': 202..."
1,919428,30083.0,157.169.233.89,2023-12-10 04:13:50,"{'id': 541654, 'action': 'search', 'time': 202..."
2,630539,NaN,160.111.52.224,2023-09-26 04:09:44,"{'id': 230556, 'action': 'click', 'time': 2023..."


In [56]:
user_session.action_time.isna().sum()

13922

In [65]:
user_session.action_time = user_session.action.apply(lambda x: x['time'])
user_session.action_time.isna().sum()

0

In [72]:
user_session[(user_session.action_time.dt.year == 2023) & (user_session.action_time.dt.month == 8)].count()['id']

8395

# Исследование данных

## Задание 1

In [73]:
def round_to_2(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.
    
    Аргументы:
        x: Число.
        
    Возвращаемое значение:
        Результат округления числа до 2 знаков после запятой.
    """
    
    return round(x, 2)

In [74]:
def get_sessions(action_times, max_delta=40 * 60):
    """
    Разбивает список моментов времени, когда пользователь проявлял активность,
    на пользовательские сессии. 
    
    Аргументы:
        action_times: Список моментов времени, когда пользователь проявлял активность.
                      Гарантируется, что моменты времени упорядочены в порядке возрастания.
           max_delta: Максимальное значение (в секундах) промежутка между двумя активностями пользователя,
                      при котором они считаются относящимися к одной сессии.
                      
    Возвращаемое значение:
        Список пользовательских сессий. Каждая сессия представляется упорядоченным по возрастанию
        списком моментов времени для действий, которые были совершены в эту сессию.
    """
    
    sessions = []
    cur_session = []
    
    prev_time = None
    
    for time in action_times:
        if prev_time is None or (time - prev_time).total_seconds() > max_delta:
            if len(cur_session) > 0:
                sessions.append(cur_session)
                
            cur_session = [time]
        else:
            cur_session.append(time)
            
        prev_time = time
        
    sessions.append(cur_session)
    
    return sessions

In [127]:
def get_avg_session_time(action_times):
    """
    По списку моментов времени, когда пользователь проявлял активность,
    вычисляет среднюю продолжительность его пользовательской сессии
    с точки зрения времени.
    
    Аргументы:
        action_times: Список моментов времени, когда пользователь проявлял активность.
                      Гарантируется, что моменты времени упорядочены в порядке возрастания.
                      
    Возвращаемое значение:
        Средняя продолжительность пользовательской сессии **в секундах**,
        округлённая до $2$-х знаков после запятой.
    """
    sesions_duration = []
    for session in get_sessions(action_times):
        if len(session) > 1:
            sesions_duration.append((session[-1] - session[0]).total_seconds())
    return round_to_2((sum(sesions_duration) / len(sesions_duration)) / 60) if sesions_duration else 0

In [128]:
def get_avg_session_time_test():
    example_1_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=41, second=21),
        pd.Timestamp(year=2023, month=1, day=13, hour=15, minute=41, second=21),
    ]
    
    example_1_res = 23.77
    
    custom_compare(get_avg_session_time(example_1_action_times), example_1_res)
    
    example_2_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=13, hour=15, minute=41, second=21),
    ]
    
    example_2_res = 0
    
    custom_compare(get_avg_session_time(example_2_action_times), example_2_res)
    
    example_3_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=18, second=24),
        pd.Timestamp(year=2023, month=2, day=21, hour=9, minute=42, second=31),
        pd.Timestamp(year=2023, month=2, day=21, hour=10, minute=21, second=9),
        pd.Timestamp(year=2023, month=2, day=21, hour=10, minute=37, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=17, minute=37, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=18, minute=12, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=18, minute=44, second=53),
        pd.Timestamp(year=2023, month=2, day=27, hour=19, minute=2, second=11),
        pd.Timestamp(year=2023, month=8, day=21, hour=10, minute=37, second=46),
    ]
    
    example_3_res = 46.83
    
    custom_compare(get_avg_session_time(example_3_action_times), example_3_res)
    
    print('Все тесты прошли успешно!')

In [129]:
get_avg_session_time_test()

Все тесты прошли успешно!


## Задание 2

Используя функцию `get_avg_session_time`, вычислите среднюю продолжительность пользовательской сессии для пользователей из таблицы `user_session`, с которой вы работали в рамках предыдущих упражнений. Для этого усредните значения средней продолжительности пользовательских сессий по всем пользователям.

**Обратите внимание**, что среднее можно вычислять как по всем пользователям вообще, так и по тем, у кого среднее время пользовательской сессии больше $0$. В рамках задания вычислите значение по всему множеству пользователей.

Аналогично лекции нужно считать, что пользователя уникальным образом идентифицирует IP-адрес, с которого он совершал действия.

Ответ округлите до $2$-х знаков после запятой.

In [145]:
avg_sessions = []
for ip, group in user_session.sort_values('action_time', ascending=True).groupby('ip'):
    avg_sessions.append(get_avg_session_time(group['action_time']))
round_to_2(sum(avg_sessions) / len(avg_sessions))

3.38

## Задание 3

### Вопрос 1

У какой доли пользователей среднее значение пользовательской сессии равно $0$?

Ответ округлите до $2$-х знаков после запятой.

In [146]:
zero_lenght_sessions = 0
for session in avg_sessions:
    if session == 0:
        zero_lenght_sessions += 1
round_to_2(zero_lenght_sessions / len(avg_sessions))

0.4

### Вопрос 2

Рассчитайте средннюю продолжительность пользовательской сессии только по пользователям, у которых средняя продолжительность пользовательской сессии больше $0$.

Ответ округлите до $2$-х знаков после запятой.

In [147]:
full_sessions = []
for session in avg_sessions:
    if session != 0:
        full_sessions.append(session)
round_to_2(sum(full_sessions) / len(full_sessions))

5.62